In [1]:
!nvidia-smi

Wed Nov 27 01:13:29 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070 ...    Off |   00000000:01:00.0  On |                  N/A |
| N/A   84C    P0             46W /   80W |    1003MiB /   8192MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
#%pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12
#%pip install cucim-cu12 cupy-cuda12x

In [3]:

import pandas as pd
import tensorflow as tf

import glob
import re

2024-11-27 01:13:30.944847: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-27 01:13:31.020471: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-27 01:13:31.042027: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 01:13:31.171640: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
base_path = '/home/workspace/geometry-classifier/data/'



In [5]:
import pandas as pd
import tensorflow as tf
import glob
import os

def serialize_example(name, image_data):
    """Serialize data into TFRecord format."""
    feature = {
        'name': tf.train.Feature(bytes_list=tf.train.BytesList(value=[name])),
        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_data]))
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def parquet_to_tfrecord(parquet_file, tfrecord_file):
    """Convert a single Parquet file to a TFRecord file."""
    df = pd.read_csv(parquet_file).drop(columns=["id"])
    
    with tf.io.TFRecordWriter(tfrecord_file, options='GZIP') as writer:
        for _, row in df.iterrows():
            name = row['name'].astype('float32').tobytes()  # Convert uint8 name to bytes
            image_data = row.drop('name').to_numpy().astype('float32').tobytes()  # Flatten image data and convert to bytes
            example = serialize_example(name, image_data)
            writer.write(example)


def convert(file_type:str, files: list):
    i=0
    for parquet_file in files:
        tfrecord_file = f"{base_path}tfr_strat100-2min/{file_type}{i}.tfrecord.gz"
        parquet_to_tfrecord(parquet_file, tfrecord_file)
        print(f"Converted {parquet_file} to {tfrecord_file}")
        i+=1

base2 = base_path + 'tfr_strat100-1min'

In [6]:
convert('test', glob.glob(f"{base2}/test*.csv"))


Converted /home/workspace/geometry-classifier/data/tfr_strat100-1min/test_150.csv to /home/workspace/geometry-classifier/data/tfr_strat100-2min/test0.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/tfr_strat100-1min/test_135.csv to /home/workspace/geometry-classifier/data/tfr_strat100-2min/test1.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/tfr_strat100-1min/test_155.csv to /home/workspace/geometry-classifier/data/tfr_strat100-2min/test2.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/tfr_strat100-1min/test_8.csv to /home/workspace/geometry-classifier/data/tfr_strat100-2min/test3.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/tfr_strat100-1min/test_146.csv to /home/workspace/geometry-classifier/data/tfr_strat100-2min/test4.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/tfr_strat100-1min/test_100.csv to /home/workspace/geometry-classifier/data/tfr_strat100-2min/test5.tfrecord.gz
Converted /home/workspace/geom

In [7]:
convert('val', glob.glob(f"{base2}/val*.csv"))

Converted /home/workspace/geometry-classifier/data/tfr_strat100-1min/val_184.csv to /home/workspace/geometry-classifier/data/tfr_strat100-2min/val0.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/tfr_strat100-1min/val_167.csv to /home/workspace/geometry-classifier/data/tfr_strat100-2min/val1.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/tfr_strat100-1min/val_228.csv to /home/workspace/geometry-classifier/data/tfr_strat100-2min/val2.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/tfr_strat100-1min/val_165.csv to /home/workspace/geometry-classifier/data/tfr_strat100-2min/val3.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/tfr_strat100-1min/val_283.csv to /home/workspace/geometry-classifier/data/tfr_strat100-2min/val4.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/tfr_strat100-1min/val_190.csv to /home/workspace/geometry-classifier/data/tfr_strat100-2min/val5.tfrecord.gz
Converted /home/workspace/geometry-class

In [8]:
train_glob = glob.glob(f"{base2}/train*.csv")
print(train_glob)
convert('train', train_glob)


['/home/workspace/geometry-classifier/data/tfr_strat100-1min/train_365.csv', '/home/workspace/geometry-classifier/data/tfr_strat100-1min/train_667.csv', '/home/workspace/geometry-classifier/data/tfr_strat100-1min/train_744.csv', '/home/workspace/geometry-classifier/data/tfr_strat100-1min/train_694.csv', '/home/workspace/geometry-classifier/data/tfr_strat100-1min/train_585.csv', '/home/workspace/geometry-classifier/data/tfr_strat100-1min/train_374.csv', '/home/workspace/geometry-classifier/data/tfr_strat100-1min/train_394.csv', '/home/workspace/geometry-classifier/data/tfr_strat100-1min/train_579.csv', '/home/workspace/geometry-classifier/data/tfr_strat100-1min/train_322.csv', '/home/workspace/geometry-classifier/data/tfr_strat100-1min/train_739.csv', '/home/workspace/geometry-classifier/data/tfr_strat100-1min/train_762.csv', '/home/workspace/geometry-classifier/data/tfr_strat100-1min/train_379.csv', '/home/workspace/geometry-classifier/data/tfr_strat100-1min/train_599.csv', '/home/work